In [3]:
using ITensors
using ITensors: position!
using ITensorTDVP
using KrylovKit
using ProgressMeter

#ProgressMeter.ijulia_behavior(:append)

In [5]:
struct TDVP2 end

singlesite!(PH::ProjMPO) = (PH.nsite = 1)
twosite!(PH::ProjMPO) = (PH.nsite = 2)

function measure_Sz(psi,n)
    psi = ITensors.orthogonalize(psi,n)
    sn = siteind(psi,n)
    Sz = scalar(dag(prime(psi[n],"Site"))*op("Sz",sn)*psi[n])
    return real(Sz)
end


function tdvp2!(ψ,H::MPO,dt,tf; kwargs...)
    
    num_time_steps = Int(tf/dt)
    
    #kwargs
   
    #The error tolerance for `KrylovKit.exponentiate`
    exp_tol = get(kwargs,:exp_tol, 1e-14)
    #Passed to KrylovKit
    krylovdim = get(kwargs,:krylovdim, 30 )
    maxiter = get(kwargs,:maxiter,100)
    normalize = get(kwargs,:normalize,true)
    #Is the Hamiltonian H hermitian? Useful to KrylovKit for exponentiation
    hermitian = get(kwargs,:hermitian,true)
    
    #If progress bar is used
    pbar = get(kwargs,:progress, true) ? Progress(num_time_steps, desc="Evolving state... ") : nothing
    
    #Imaginary time step
    τ = 1im*dt
    
    num_sites = length(ψ)
    #start with right orthogonalised wave function
    ITensors.orthogonalize!(ψ,1)
    
    #https://docs.juliahub.com/ITensors/P3pqL/0.1.7/ProjMPO.html
    PH = ProjMPO(H)
    
    #Projected MPO
    
    position!(PH,ψ,1)
    
    
    #Do the time evolution
    for time_step in 1:num_time_steps
        stime  = @elapsed begin
        # b is the active site  ha gives 1/2 depending on right/left swap
        for (b,ha) in sweepnext(N)
                # 2 - site TDVP .
                twosite!(PH)

                
                #orthogonalise at position b
                ITensors.position!(PH,ψ,b)
                wf = ψ[b]*ψ[b+1]
                wf, info = exponentiate(PH, -τ/2, wf; ishermitian=hermitian , tol=exp_tol, krylovdim=krylovdim)
                #Determine sweep direction
                dir = ha==1 ? "left" : "right"
                info.converged==0 && throw("exponentiate did not converge")
                #Put back the updated ITensor(wf) in ψ
                spec = replacebond!(ψ,b,wf;normalize=normalize, ortho = dir, kwargs... )
                
                
               
        

            # evolve with single-site Hamiltonian backward in time.
            # In the case of imaginary time-evolution this step
            # is not necessary (see Ref. [1])
            i = ha==1 ? b+1 : b
            if 1<i<N && !(dt isa Complex)
                singlesite!(PH)

                ITensors.position!(PH,ψ,i)
                ψ[i], info = exponentiate(PH,τ/2,ψ[i]; ishermitian=hermitian, tol=exp_tol, krylovdim=krylovdim,
                                            maxiter=maxiter)
                info.converged==0 && throw("exponentiate did not converge")
            elseif i==1 && dt isa Complex
                # TODO not sure if this is necessary anymore
                ψ[i] /= sqrt(real(scalar(dag(ψ[i])*ψ[i])))
            end

        end
        end
        
        
        Sz = measure_Sz(ψ,5)
        !isnothing(pbar) && ProgressMeter.next!(pbar, showvalues=[("t", dt*time_step),
                                                                  ("dt step time", round(stime,digits=3)),
                                                                  ("Max bond-dim", maxlinkdim(ψ)),
                                                                  ("Sz @ site 5",Sz )])
        
    
    
        
    end
end


tdvp2! (generic function with 1 method)

In [8]:
for (b,ha) in sweepnext(N)
    println(b," ",ha)
end
    

1 1
2 1
3 1
4 1
5 1
6 1
7 1
8 1
9 1
9 2
8 2
7 2
6 2
5 2
4 2
3 2
2 2
1 2


In [ ]:
function tdvp1!(ψ,H::MPO,dt,tf; kwargs...)
    
    num_time_steps = Int(tf/dt)
    
    #kwargs
   
    #The error tolerance for `KrylovKit.exponentiate`
    exp_tol = get(kwargs,:exp_tol, 1e-14)
    #Passed to KrylovKit
    krylovdim = get(kwargs,:krylovdim, 30 )
    maxiter = get(kwargs,:maxiter,100)
    normalize = get(kwargs,:normalize,true)
    #Is the Hamiltonian H hermitian? Useful to KrylovKit for exponentiation
    hermitian = get(kwargs,:hermitian,true)
    
    #If progress bar is used
    pbar = get(kwargs,:progress, true) ? Progress(num_time_steps, desc="Evolving state... ") : nothing
    
    #Imaginary time step
    τ = 1im*dt
    
    num_sites = length(ψ)
    #start with right orthogonalised wave function
    ITensors.orthogonalize!(ψ,1)
    
    #https://docs.juliahub.com/ITensors/P3pqL/0.1.7/ProjMPO.html
    PH = ProjMPO(H)
    
    #Projected MPO
    
    position!(PH,ψ,1)
    
    
    #Do the time evolution
    for time_step in 1:num_time_steps
        stime  = @elapsed begin
        # b is the active site  ha gives 1/2 depending on right/left swap
        for (b,ha) in sweepnext(N)
                # 1 - site TDVP .
                singlesite!(PH)

                
                #orthogonalise at position b
                ITensors.position!(PH,ψ,b)
                wf = ψ[b]
                wf, info = exponentiate(PH, -τ/2, wf; ishermitian=hermitian , tol=exp_tol, krylovdim=krylovdim)
                U,S,V = svd(M,i) 
                C=S*V
                #Determine sweep direction
                dir = ha==1 ? "left" : "right"
                info.converged==0 && throw("exponentiate did not converge")
                #Put back the updated ITensor(wf) in ψ
                spec = replacebond!(ψ,b,U;normalize=normalize, ortho = dir, kwargs... )
                
                
               
        

            # evolve with single-site Hamiltonian backward in time.
            # In the case of imaginary time-evolution this step
            # is not necessary (see Ref. [1])
            i = ha==1 ? b+1 : b
            if 1<i<N && !(dt isa Complex)
                singlesite!(PH)

                ITensors.position!(PH,ψ,i)
                ψ[i], info = exponentiate(PH,τ/2,ψ[i]; ishermitian=hermitian, tol=exp_tol, krylovdim=krylovdim,
                                            maxiter=maxiter)
                info.converged==0 && throw("exponentiate did not converge")
            elseif i==1 && dt isa Complex
                # TODO not sure if this is necessary anymore
                ψ[i] /= sqrt(real(scalar(dag(ψ[i])*ψ[i])))
            end

        end
        end
        
        
        Sz = measure_Sz(ψ,5)
        !isnothing(pbar) && ProgressMeter.next!(pbar, showvalues=[("t", dt*time_step),
                                                                  ("dt step time", round(stime,digits=3)),
                                                                  ("Max bond-dim", maxlinkdim(ψ)),
                                                                  ("Sz @ site 5",Sz )])
        
    
    
        
    end
end

In [6]:
N = 10
cutoff = 1e-10

s = siteinds("S=1/2", N)

os = OpSum()
for j in 1:(N - 1)
    os += 0.5, "S+", j, "S-", j + 1
    os += 0.5, "S-", j, "S+", j + 1
    os += "Sz", j, "Sz", j + 1
end

H = MPO(os, s)
ψ = productMPS(s, n -> isodd(n) ? "Up" : "Dn")
ψ1 = copy(ψ)

MPS
[1] ((dim=2|id=107|"S=1/2,Site,n=1"), (dim=1|id=522|"Link,l=1"))
[2] ((dim=1|id=522|"Link,l=1"), (dim=2|id=189|"S=1/2,Site,n=2"), (dim=1|id=499|"Link,l=2"))
[3] ((dim=1|id=499|"Link,l=2"), (dim=2|id=533|"S=1/2,Site,n=3"), (dim=1|id=379|"Link,l=3"))
[4] ((dim=1|id=379|"Link,l=3"), (dim=2|id=544|"S=1/2,Site,n=4"), (dim=1|id=619|"Link,l=4"))
[5] ((dim=1|id=619|"Link,l=4"), (dim=2|id=707|"S=1/2,Site,n=5"), (dim=1|id=832|"Link,l=5"))
[6] ((dim=1|id=832|"Link,l=5"), (dim=2|id=518|"S=1/2,Site,n=6"), (dim=1|id=448|"Link,l=6"))
[7] ((dim=1|id=448|"Link,l=6"), (dim=2|id=995|"S=1/2,Site,n=7"), (dim=1|id=138|"Link,l=7"))
[8] ((dim=1|id=138|"Link,l=7"), (dim=2|id=498|"S=1/2,Site,n=8"), (dim=1|id=924|"Link,l=8"))
[9] ((dim=1|id=924|"Link,l=8"), (dim=2|id=641|"S=1/2,Site,n=9"), (dim=1|id=131|"Link,l=9"))
[10] ((dim=1|id=131|"Link,l=9"), (dim=2|id=555|"S=1/2,Site,n=10"))


In [7]:
tdvp2!(ψ1,H,0.1,5.0,exp_tol=cutoff)

┌ Warning: ProgressMeter by default refresh meters with additional information in IJulia via `IJulia.clear_output`, which clears all outputs in the cell. 
│  - To prevent this behaviour, do `ProgressMeter.ijulia_behavior(:append)`. 
│  - To disable this warning message, do `ProgressMeter.ijulia_behavior(:clear)`.
└ @ ProgressMeter /home/sandipanmanna/.julia/packages/ProgressMeter/sN2xr/src/ProgressMeter.jl:618
Evolving state... 100%|█████████████████████████████████| Time: 0:00:47
  t:             5.0
  dt step time:  0.44
  Max bond-dim:  32
  Sz @ site 5:   0.00739252560015288


In [9]:
ϕ = tdvp(
  H,
  -1.0,
  ψ;
  nsweeps=20,
  reverse_step=false,
  normalize=true,
  maxdim=30,
  cutoff=1e-10,
  outputlevel=1,
) 

After sweep 1: maxlinkdim=4 maxerr=8.89E-17 current_time=-1.0 time=15.676
After sweep 2: maxlinkdim=16 maxerr=8.37E-11 current_time=-2.0 time=0.054
After sweep 3: maxlinkdim=20 maxerr=7.41E-11 current_time=-3.0 time=0.068
After sweep 4: maxlinkdim=20 maxerr=7.80E-11 current_time=-4.0 time=0.066
After sweep 5: maxlinkdim=20 maxerr=7.81E-11 current_time=-5.0 time=0.066
After sweep 6: maxlinkdim=20 maxerr=7.81E-11 current_time=-6.0 time=0.053
After sweep 7: maxlinkdim=20 maxerr=7.81E-11 current_time=-7.0 time=0.043
After sweep 8: maxlinkdim=20 maxerr=7.81E-11 current_time=-8.0 time=0.032
After sweep 9: maxlinkdim=20 maxerr=7.81E-11 current_time=-9.0 time=0.028
After sweep 10: maxlinkdim=20 maxerr=7.81E-11 current_time=-10.0 time=0.036
After sweep 11: maxlinkdim=20 maxerr=7.81E-11 current_time=-11.0 time=0.024
After sweep 12: maxlinkdim=20 maxerr=7.81E-11 current_time=-12.0 time=0.024
After sweep 13: maxlinkdim=20 maxerr=7.81E-11 current_time=-13.0 time=0.034
After sweep 14: maxlinkdim=20 

MPS
[1] ((dim=2|id=197|"Link,l=1"), (dim=2|id=351|"S=1/2,Site,n=1"))
[2] ((dim=4|id=119|"Link,l=2"), (dim=2|id=169|"S=1/2,Site,n=2"), (dim=2|id=197|"Link,l=1"))
[3] ((dim=8|id=639|"Link,l=3"), (dim=2|id=676|"S=1/2,Site,n=3"), (dim=4|id=119|"Link,l=2"))
[4] ((dim=16|id=225|"Link,l=4"), (dim=2|id=105|"S=1/2,Site,n=4"), (dim=8|id=639|"Link,l=3"))
[5] ((dim=20|id=616|"Link,l=5"), (dim=2|id=709|"S=1/2,Site,n=5"), (dim=16|id=225|"Link,l=4"))
[6] ((dim=16|id=897|"Link,l=6"), (dim=2|id=342|"S=1/2,Site,n=6"), (dim=20|id=616|"Link,l=5"))
[7] ((dim=8|id=600|"Link,l=7"), (dim=2|id=438|"S=1/2,Site,n=7"), (dim=16|id=897|"Link,l=6"))
[8] ((dim=4|id=939|"Link,l=8"), (dim=2|id=597|"S=1/2,Site,n=8"), (dim=8|id=600|"Link,l=7"))
[9] ((dim=2|id=719|"Link,l=9"), (dim=2|id=611|"S=1/2,Site,n=9"), (dim=4|id=939|"Link,l=8"))
[10] ((dim=2|id=884|"S=1/2,Site,n=10"), (dim=2|id=719|"Link,l=9"))


In [10]:
print(inner(ψ1,ϕ))

-0.279608979839224 + 0.1065337650325162im